In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
import xgboost as xg
from tqdm import tqdm
from sklearn.metrics import mean_squared_error as MSE
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
import os
import PIL.Image
import cv2
import matplotlib.pyplot as plt
from collections import Counter
import pickle

In [2]:
parent_folder = './'

In [3]:
labels_df = pd.read_csv(parent_folder+'staged_biopsy_holdout.csv')
sort_idx = np.argsort( labels_df.slide_id.values )
labels_df = labels_df.loc[sort_idx]
labels_df.reset_index(inplace=True, drop=True)
gby_temp = list(labels_df.groupby('biopsy_id',sort=False))
biopsy_idx_to_slide_idx = np.array( [ l[1].index.values for l in gby_temp ], dtype=object )

In [4]:
labels_df.head()

,Unnamed: 0,slide_id,biopsy_id
0,2537,000a3de5-c8b9-4f53-8d15-07d76894eca7,4b6d2dfd-615a-48b5-99f6-952f9b726b1e
1,13945,000ed373-18a4-400b-811c-dd432dc90dc2,013d29dc-b4ed-48f5-985c-2793b3071d8c
2,5278,000f344e-fb53-41f8-b6a9-e8e2ec41a204,4a5d36b0-1047-4dd9-b041-f102c11493c4
3,16042,00157adc-75db-4ac8-9237-adbf6df6a3de,e0d293bf-76ab-4b07-85a3-c7a0a38d2ec6
4,11533,001a0d82-f168-4f3b-bad3-0ab5afca7d6f,c152f054-46f2-4921-bcb8-ea1fcf149220


In [5]:
len(labels_df["slide_id"].value_counts())

14487

In [6]:
len(labels_df["biopsy_id"].value_counts())

886

In [7]:
# for each biopsy make a list of the slides id's by re-shaping the df. 
list_df = labels_df.groupby(["biopsy_id"]).agg({'slide_id': lambda x: x.tolist()}).reset_index()
list_df.head()

,biopsy_id,slide_id
0,0046d594-872b-47ce-bdea-b2853f249a4a,"[15542234-4161-4af3-a659-2a8815e9283a, 1cad8f8..."
1,0048fb50-689b-432d-acc6-9be1d30ddaba,"[19090436-05c4-465b-b111-0ba6a1494723, 1f707ce..."
2,00acb063-63a1-4241-9a1a-a824c783782e,"[139f770a-c524-447c-91bd-fdc4fd4d738d, 1a18f15..."
3,00d0a1e5-0ee6-4a17-9ab1-7cf14faab778,"[70cc82cb-3d32-43cc-8ac8-263136ce0ff1, 9ae7857..."
4,013d29dc-b4ed-48f5-985c-2793b3071d8c,"[000ed373-18a4-400b-811c-dd432dc90dc2, 0341292..."


In [60]:
list_df.shape

(2721, 2)

In [9]:
def re_pack(x):
    biopsy_bag = np.zeros((1,2048))
    for slide in x["slide_id"]:
        try:
            slide_bag = np.load("../save_resnet_embeddings_level4_bags_bracs/resnet50_bracs_"+slide+".npy")
            biopsy_bag = np.vstack((biopsy_bag,slide_bag)).astype(np.float16)
            # append to biopsy bag.
        except:
            pass
    # save biopsy bag with biopsy id as the name.
    biopsy_bag = biopsy_bag[1:,:]
    np.save("../save_resnet_embeddings_level4_biopsy_bags_bracs_float16/"+x["biopsy_id"]+".npy",biopsy_bag)
     
# lambda function for entire set of all bags.
list_df.apply(lambda x: re_pack(x) ,axis=1) # uncomment top do the packing!


0      None
1      None
2      None
3      None
4      None
       ... 
881    None
882    None
883    None
884    None
885    None
Length: 886, dtype: object

In [136]:
list_df["biopsy_id"].to_csv("HOLD_OUT.csv")